## 프로젝트명 : 머신러닝을 활용한 기업부실예측모형

### 0. 환경설정

#### 1) 폴더 생성 : 사전에 C 드라이브에 폴더 생성하여 주피터노트북 실행 
#### 2) 쥬피토노트북 실행 : 명령프롬프트- C:\finance_data>jupyter notebook
#### 3) Code & data 자료 : https://github.com/freejyb/DT --> 프로젝트실습
#### 4) 금융감독원 전저공시시스템(Dart) : https://dart.fss.or.kr/
#### (참고) 미국 sec : https://www.sec.gov/edgar/search/

### Project 기획서 : Work Flow

1. 데이터 수집
- 대상기업 : 상장기업 부실기업(1), 정상기업(0), 12월 결산 기업 금융업 제외
- 데이터 수집처 : DART
- 데이터 수집 : 재무제표 관련 재무비율 생성(feature): 안정성, 성장성, 수익성, 활동성
- 데이터 수집기간 : 2015~2023년

2. 데이터 전처리
- 결측치(Missing Values), 이상치(Outliers) 처리
- 데이터 분할 :
- 1) 훈련 데이터(Training Set): 모델 학습에 사용.
- 2) 검증 데이터(Validation Set): 모델의 튜닝 및 성능 평가에 사용.
- 3) 테스트 데이터(Test Set): 
3. EDA (Exploratory Data Analysis) 및 시각화 (Visualization)
- 기초통계량
- 데이터 확인
- 시각화 

4. Feature selction : 재무비율(후보 20개)
- LASSO (Least Absolute Shrinkage and Selection Operator)
- 후진제거법 (Backward Elimination)
5. 모델링 : 분류모델
- Logistic Regression
- Decision Tree
- Random Forest
- XGBoost (Extreme Gradient Boosting)
6. 성능 개선
- Hyperparameter 조정
- Resampling (over sampleing)
- Cross-Validation: CV

### 1. Data Set

#### 1-1 데이터수집하기 (Raw data)

- https://opendart.fss.or.kr/disclosureinfo/fnltt/dwld/main.do
- 싸이트 접속- 연도별별로 f/s 다운로드:2024년도를 제외한 2015년도부터 2023년까지
- F/S 3가지 : 사업보고서 재무상태표, 손익계산서, 현금흐름표
- 다운받은 파일은 zip파일(확장자. txt)로 되어있어 각 년도의 압축 파일 중 금융을 제외한 "년도_사업보고서_01_재무상태표_","년도_사업보고서_01_재무상태표_연결"로 되어있는 데이터를 저장(현금흐름표,재무상태표)
- 손익계산서는 년도_사업보고서_03_포괄손익계산서, 년도_사업보고서_03_포괄손익계산서_연결 사용
- 
- 사용자 폴더에 저장 : 폴더명..... 폴더안에 텍스트data 3가지x 2(연결, 개별) x 9(9개년도) = 54개 파일

#### 1-2 데이터 합치기

- 각각 연결데이터와 개별데이터를 융합
- 연결데이터를 쓰기위해 연결데이터에 빈값은 개별데이터로 채우는 과정이 필요하다.

In [1]:
# 코드 설명: 라이브러리 임포트
# os 모듈: 파일 및 디렉터리 조작을 위한 환경 설정.
# pandas 라이브러리: 데이터프레임을 활용한 데이터 분석 도구 로드.
# warnings.filterwarnings("ignore"): 불필요한 경고를 숨겨 작업 환경을 깔끔하게 유지.

In [2]:
# 폴더 경고 설정
# folder_path 변수
# 데이터가 저장된 폴더 경로를 저장하는 문자열 변수입니다.
# 이후 코드에서 데이터 파일을 불러오거나 처리할 때 이 경로를 참조합니다.
# ./ (현재 디렉터리)
# ./는 **현재 작업 디렉터리(current working directory)**를 나타냅니다.
# 예를 들어, 스크립트가 /project 폴더에서 실행 중이라면 ./원본 데이터는 가리킵니다.
# data/원본 데이터

# 현재 디렉터리 아래에 있는 data 폴더의 하위 폴더 원본 데이터를 의미합니다.
# 데이터가 해당 경로에 저장되어 있어야 코드가 정상적으로 작동합니다.

In [3]:
# os.listdir(folder_path)
# 지정된 폴더(folder_path) 안에 있는 모든 파일과 디렉터리의 이름을 리스트로 반환합니다.

# os.listdir(folder_path)로 반환된 파일 리스트에서 확장자가 .txt인 파일만 필터링합니다.
# file.endswith('.txt')는 파일 이름이 .txt로 끝나는지 확인하는 조건입니다.
# 결과적으로, 폴더 내의 텍스트 파일만 포함된 리스트를 생성합니다.

# print(txt_files)
# 텍스트 파일 목록을 출력합니다.

In [4]:
## 연결재무제표 재무제표 구분
# connected_files = [file for file in txt_files if '연결' in file]
# non_connected_files = [file for file in txt_files if '연결' not in file]
# 목적: 텍스트 파일 중 **'연결'**이라는 문자열을 포함하는 파일과 포함하지 않는 파일을 분리합니다.
# 구분 기준:
# '연결' in file: 파일 이름에 '연결'이라는 단어가 포함된 경우.
# '연결' not in file: 파일 이름에 '연결'이라는 단어가 포함되지 않은 경우.
# 리스트 컴프리헨션:

# # 데이터를 담을 리스트 초기화
# connected_files: 파일 이름에 '연결'이 포함된 파일만 선택해서 저장.
# non_connected_files: 파일 이름에 '연결'이 포함되지 않은 파일만 선택해서 저장.
# 목적: 이후에 파일에서 읽어들인 데이터를 저장할 빈 리스트를 생성합니다.


In [5]:
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 폴더 경로 설정
folder_path = './원본 데이터'  # 폴더 경로 수정 필요

# 폴더 내의 모든 텍스트 파일(.txt) 가져오기
txt_files = [file for file in os.listdir(folder_path) if file.endswith('.txt')]
# 파일 목록 출력
print(txt_files)

# 연결재무제표 재무제표 구분(변수명 새로 설정)
connected_files = [file for file in txt_files if '연결' in file]
non_connected_files = [file for file in txt_files if '연결' not in file]

# 데이터를 담을 리스트 초기화
connected_data = []
non_connected_data = []

['2015_사업보고서_01_재무상태표_20230503.txt', '2015_사업보고서_01_재무상태표_연결_20230503.txt', '2015_사업보고서_03_포괄손익계산서_20230503.txt', '2015_사업보고서_03_포괄손익계산서_연결_20230503.txt', '2015_사업보고서_04_현금흐름표_20230503.txt', '2015_사업보고서_04_현금흐름표_연결_20230503.txt', '2016_사업보고서_01_재무상태표_20241115.txt', '2016_사업보고서_01_재무상태표_연결_20241115.txt', '2016_사업보고서_03_포괄손익계산서_20241115.txt', '2016_사업보고서_03_포괄손익계산서_연결_20241115.txt', '2016_사업보고서_04_현금흐름표_20241115.txt', '2016_사업보고서_04_현금흐름표_연결_20241115.txt', '2017_사업보고서_01_재무상태표_20241115.txt', '2017_사업보고서_01_재무상태표_연결_20241115.txt', '2017_사업보고서_03_포괄손익계산서_20241115.txt', '2017_사업보고서_03_포괄손익계산서_연결_20241115.txt', '2017_사업보고서_04_현금흐름표_20241115.txt', '2017_사업보고서_04_현금흐름표_연결_20241115.txt', '2018_사업보고서_01_재무상태표_20241115.txt', '2018_사업보고서_01_재무상태표_연결_20241115.txt', '2018_사업보고서_03_포괄손익계산서_20241115.txt', '2018_사업보고서_03_포괄손익계산서_연결_20241115.txt', '2018_사업보고서_04_현금흐름표_20241115.txt', '2018_사업보고서_04_현금흐름표_연결_20241115.txt', '2019_사업보고서_01_재무상태표_20241115.txt', '2019_사업보고서_01_재무상태표_연결_20241115.txt', '2019_사업

In [6]:
# connected_files 리스트에 있는 '연결' 재무제표 파일들을 읽고, 
# 내용을 **Pandas 데이터프레임(DataFrame)**으로 변환하여 connected_data 리스트에 저장하는 과정입니다.

# os.path.join: 폴더 경로(folder_path)와 파일 이름(file_name)을 결합하여 전체 파일 경로를 생성합니다.
# 예: folder_path = './data', file_name = '연결_재무제표_2023.txt'
# 결과: './data/연결_재무제표_2023.txt'

# # 파일 읽기
# pd.read_csv: 파일을 Pandas 데이터프레임으로 읽어오는 함수.
# file_path: 읽어올 파일 경로.
# encoding='cp949': 한글이 포함된 파일에 적합한 인코딩 방식. 일반적으로 Windows에서 생성된 파일에서 사용.
# delimiter='\t': 파일이 **탭(tab)**으로 구분된 형식일 때 이를 명시.
# 탭은 CSV(Comma-Separated Values) 대신 TSV(Tab-Separated Values) 형식에서 사용됨.

# 데이터 추가
# 읽어들인 데이터프레임(df)을 connected_data 리스트에 추가.
# 결과적으로, connected_data에는 모든 연결 재무제표 파일의 데이터프레임이 리스트 형태로 저장됩니다.

# except Exception as e:
#     print(f"Error reading file {file_name}: {e}")
# 파일 읽기 중 오류가 발생하면, 오류를 무시하고 어떤 파일에서 문제가 발생했는지 출력.


In [7]:
# non_connected_files 리스트에 있는 **'비연결' 재무제표 파일(별도재무제표)**을 읽고, 
# 그 내용을 데이터프레임으로 변환하여 non_connected_data 리스트에 저장합니다. 
# 연결 재무제표 파일 처리와 동일한 방식으로 동작하지만, 이번에는 '비연결' 파일을 대상으로 실행됩니다.

# file_path = os.path.join(folder_path, file_name)
# 목적: folder_path(폴더 경로)와 file_name(파일 이름)을 결합해 파일의 전체 경로를 생성.
# 예: folder_path = './data', file_name = '별도_재무제표_2023.txt'
# 생성된 파일 경로: './data/별도_재무제표_2023.txt'

# df = pd.read_csv(file_path, encoding='cp949', delimiter='\t')
# Pandas read_csv 함수를 사용해 파일을 데이터프레임으로 읽어옵니다.
# encoding='cp949': 한글 파일을 읽기 위한 인코딩 지정.
# delimiter='\t': 파일이 **탭(Tab)**으로 구분된 형식일 때, 이를 명시.
# 읽은 데이터는 Pandas 데이터프레임 객체로 반환됩니다.

# non_connected_data.append(df)
# 읽은 데이터프레임(df)을 non_connected_data 리스트에 추가.
# 결과적으로, 모든 비연결 재무제표 데이터를 리스트 형태로 저장.

# except Exception as e:
#     print(f"Error reading file {file_name}: {e}")
# 목적: 파일 읽기 중 발생하는 오류를 무시하고 다음 파일로 진행.
# 오류 메시지 출력: 문제가 발생한 파일 이름과 오류 원인을 표시.

#connected_data**와 non_connected_data 리스트에 저장된 데이터프레임들을 
# 각각 하나의 데이터프레임으로 병합(concatenate)하는 과정입니다.
# pd.concat 함수
# 목적: 여러 데이터프레임을 하나로 합칩니다.
# 매개변수:
# connected_data 또는 non_connected_data: 병합할 데이터프레임 리스트.
# ignore_index=True: 새로 생성된 데이터프레임에서 기존 데이터프레임의 인덱스를 무시하고 새 인덱스를 생성.
# connected_df = pd.concat(connected_data, ignore_index=True)
# connected_data: 연결 재무제표 데이터를 포함한 데이터프레임 리스트.
# 결과:# connected_data 리스트에 저장된 모든 데이터프레임을 하나의 데이터프레임(connected_df)으로 합침.
# 새로 생성된 데이터프레임은 기존 인덱스를 무시하고 새로 번호가 매겨집니다.

# 별도 재무제표 병합
# non_connected_df = pd.concat(non_connected_data, ignore_index=True)
# non_connected_data: 별도 재무제표 데이터를 포함한 데이터프레임 리스트.
# 결과:
# non_connected_data 리스트에 저장된 모든 데이터프레임을 하나의 데이터프레임(non_connected_df)으로 합침.
# 동일하게 새 인덱스가 부여됩니다.

In [8]:
# 연결재무제표 불러오기
for file_name in connected_files:
    file_path = os.path.join(folder_path, file_name)
    try:
        df = pd.read_csv(file_path, encoding='cp949', delimiter='\t')  # 탭으로 구분된 파일 읽기
        connected_data.append(df)  # 데이터프레임을 리스트에 추가
    except Exception as e:
        print(f"Error reading file {file_name}: {e}")

# 재무제표 불러오기
for file_name in non_connected_files:
    file_path = os.path.join(folder_path, file_name)
    try:
        df = pd.read_csv(file_path, encoding='cp949', delimiter='\t')  # 탭으로 구분된 파일 읽기
        non_connected_data.append(df)  # 데이터프레임을 리스트에 추가
    except Exception as e:
        print(f"Error reading file {file_name}: {e}")

# 연결 재무제표의 데이터 리스트를 하나의 데이터프레임으로 concat 합치기(연도별 파일을 합차기)
connected_df = pd.concat(connected_data, ignore_index=True)

# 개별재무제표 데이터 리스트를 하나의 데이터프레임으로 conat 합치기 (연도뱔 파일을 합차기)
non_connected_df = pd.concat(non_connected_data, ignore_index=True)


- 연결재무제표 데이터에서의 NaN값 : 개별재무제표 데이터로 채우기

In [9]:
# 병합: 회사명, 시장구분, 업종, 업종명을 기준으로 일반과 연결 데이터 병합
df_merged = pd.merge(non_connected_df, connected_df, 
                     on=["재무제표종류","종목코드","회사명", "시장구분", "업종", "업종명","결산월","결산기준일","보고서종류","통화","항목코드","항목명"], 
                     how="left", 
                     suffixes=("_개별", "_연결"))

In [10]:
# '당기_연결' 컬럼의 NaN 값을 '당기_일반데이터' 컬럼으로 채우기
df_merged['당기_연결'] = df_merged['당기_연결'].fillna(df_merged['당기_개별'])
df_merged['전기_연결'] = df_merged['전기_연결'].fillna(df_merged['전기_개별'])

In [11]:
# 재무비율 계산을 위한 필요한 컬럼만 남기기
df_fs = df_merged[["종목코드","회사명","시장구분","결산월","결산기준일","항목명",'당기_연결','전기_연결']]

# 컬럼명 수정 : 당기연결-당기, 전기_연결--> 전기
df_fs.columns = ["종목코드","회사명","시장구분","결산월","결산기준일","항목명",'당기','전기']

In [12]:
df_fs.head()

,종목코드,회사명,시장구분,결산월,결산기준일,항목명,당기,전기
0,[060310],3S,코스닥시장상장법인,3,2015-03-31,자산 [abstract],NaN,NaN
1,[060310],3S,코스닥시장상장법인,3,2015-03-31,유동자산,"14,907,177,686","16,826,799,889"
2,[060310],3S,코스닥시장상장법인,3,2015-03-31,현금및현금성자산,"6,952,652,709","4,414,124,927"
3,[060310],3S,코스닥시장상장법인,3,2015-03-31,단기금융상품,"411,483,722","526,442,079"
4,[060310],3S,코스닥시장상장법인,3,2015-03-31,매출채권 및 기타유동채권,"5,177,589,764","7,058,047,081"


In [13]:
# 전체 데이터 저장
df_fs.to_csv("./1.1 재무데이터.csv",index=None)

### 2. 후보 재무비율(feature)만들기 : 20개

- 다른 연구에서 가장 많이 적용되는 대표적 feature
- 재무비율에 한정
- Beaver, Altman, 시장모형....
- 분류모델

### 2-1 재무비율 목록(feature 후보군)

1. 유동비율 (Current Ratio)
계산식: (유동자산 / 유동부채) * 100
의미: 기업이 단기 부채를 상환할 수 있는 능력을 나타내는 지표입니다. 유동자산(현금, 매출채권 등)과 유동부채(단기 부채, 매입채무 등) 비율이 100%를 넘으면 단기 부채 상환이 가능하다는 의미로 해석됩니다.

2. 당좌비율 (Quick Ratio)
계산식: ((유동자산 - 재고자산) / 유동부채) * 100
의미: 유동자산에서 재고자산을 제외한 자산을 기준으로 단기 부채 상환 능력을 측정합니다. 재고자산은 현금화가 어려울 수 있기 때문에 제외합니다. 이 비율이 높을수록 단기 지급능력이 우수하다고 판단됩니다.

3. 부채비율 (Debt Ratio)
계산식: (총부채 / 총자산) * 100
의미: 기업의 자산 중 부채가 차지하는 비율을 나타냅니다. 이 비율이 높으면 기업이 자산을 조달하기 위해 많은 부채에 의존하고 있음을 의미하며, 재무적 위험이 클 수 있습니다.

4. 자기자본비율 (Equity Ratio)
계산식: (자기자본 / 총자산) * 100
의미: 기업이 자산을 자기자본으로 조달한 비율입니다. 이 비율이 높으면 기업의 자본금이 강하고 안정적이라는 의미입니다.

5. 차입금의존도 (Debt-to-Equity Ratio)
계산식: (총부채 / 자기자본) * 100
의미: 부채가 자기자본에 비해 얼마나 의존하는지를 나타내는 비율입니다. 비율이 높을수록 부채 의존도가 높고, 재무적 리스크가 클 수 있습니다.

6. 현금비율 (Cash Ratio)
계산식: (현금및현금성자산 / 유동부채) * 100
의미: 기업이 보유한 현금 및 현금성 자산으로 단기 부채를 상환할 수 있는 비율을 나타냅니다. 현금만으로 단기 부채를 상환할 수 있는지 평가하는 지표입니다.

7. 순운전자본비율 (Net Working Capital Ratio)
계산식: ((유동자산 - 유동부채) / 총자산) * 100
의미: 기업의 유동자산에서 유동부채를 차감한 순운전자본을 기준으로 총자산에서 차지하는 비율을 계산합니다. 순운전자본이 충분할수록 단기적인 지급능력이 높다고 볼 수 있습니다.

8. 자기자본이익률 (ROE, Return on Equity)
계산식: (순이익 / ((총자산 + 전기 총자산) / 2)) * 100
의미: 자기자본에 대한 순이익의 비율로, 기업이 자기자본을 얼마나 효율적으로 활용했는지를 평가하는 지표입니다. 이 비율이 높을수록 기업은 자기자본을 잘 활용하고 있다는 뜻입니다.

9. 총자산이익률 (ROA, Return on Assets)
계산식: (순이익 / ((총자산 + 전기 총자산) / 2)) * 100
의미: 기업이 자산을 얼마나 효율적으로 활용해 이익을 창출했는지를 나타냅니다. 높은 ROA는 자산의 효율적인 운용을 의미합니다.

10. 매출총이익률 (Gross Profit Margin)
계산식: (매출총이익 / 매출액) * 100
의미: 매출액 대비 매출총이익의 비율로, 매출에서 직접적인 비용(매출원가)을 제외한 이익을 나타냅니다. 이 비율이 높을수록 기업의 제품이나 서비스가 높은 마진을 창출하고 있다는 뜻입니다.

11. 영업이익률 (Operating Profit Margin)
계산식: (영업이익 / 매출액) * 100
의미: 매출액에서 영업비용을 제외한 후 영업이익이 차지하는 비율입니다. 영업이익률이 높을수록 기업의 본업에서 높은 수익을 올리고 있다는 의미입니다.

12. 매출액성장률 (Sales Growth Rate)
계산식: ((매출액 - 전기 매출액) / 전기 매출액) * 100
의미: 매출액의 변화율을 나타냅니다. 매출액이 증가하면 사업이 성장하고 있다는 의미로 해석할 수 있습니다.

13. 자산성장률 (Asset Growth Rate)
계산식: ((총자산 - 전기 총자산) / 전기 총자산) * 100
의미: 총자산의 변화율을 나타내며, 기업이 자산을 얼마나 확장했는지를 평가합니다.

14. 자기자본성장률 (Equity Growth Rate)
계산식: ((자기자본 - 전기 자기자본) / 전기 자기자본) * 100
의미: 자기자본의 변화율로, 기업의 자본 증가나 감소를 확인할 수 있는 지표입니다.

15. 매출총이익성장률 (Gross Profit Growth Rate)
계산식: ((매출총이익 - 전기 매출총이익) / 전기 매출총이익) * 100
의미: 매출총이익의 성장률을 나타내며, 기업의 기본적인 수익성 향상 정도를 파악할 수 있습니다.

16. 총자산회전율 (Total Asset Turnover)
계산식: (매출액 / ((총자산 + 전기 총자산) / 2)) * 100
의미: 총자산을 얼마나 효율적으로 활용했는지를 나타내는 지표입니다. 자산 대비 매출액의 비율이 높을수록 자산을 효율적으로 운용하고 있다고 볼 수 있습니다.

17. 매출채권회전율 (Receivables Turnover)
계산식: (매출액 / ((매출채권 + 전기 매출채권) / 2)) * 100
의미: 매출채권(받을 돈)이 얼마나 빠르게 현금화되는지를 나타냅니다. 이 비율이 높을수록 매출채권을 효율적으로 회전시키고 있다는 뜻입니다.

18. 재고자산회전율 (Inventory Turnover)
계산식: (매출액 / ((재고자산 + 전기 재고자산) / 2)) * 100
의미: 재고자산이 얼마나 빠르게 판매되는지를 나타냅니다. 재고를 효율적으로 회전시키고 있는지를 평가하는 지표입니다.

19. 순운전자본회전율 (Net Working Capital Turnover)
계산식: (매출액 / ((유동자산 - 유동부채) + (전기 유동자산 - 전기 유동부채)) / 2) * 100
의미: 순운전자본이 얼마나 효율적으로 매출을 창출하고 있는지를 나타냅니다. 순운전자본이 많을수록 더 많은 자산을 운용할 수 있습니다.

20. 총부채상환능력비율 (Debt Repayment Ability Ratio)
계산식: (영업활동현금흐름 / 총부채) * 100
의미: 영업활동을 통해 얻은 현금흐름으로 얼마나 부채를 상환할 수 있는지를 나타냅니다. 이 비율이 높을수록 부채 상환 능력이 뛰어난 기업이라고 볼 수 있습니다.

In [14]:
## '기업명' 컬럼의 띄어쓰기를 제거 : 일치
df_fs['항목명'] = df_fs['항목명'].str.replace(' ', '', regex=False)
df_fs['항목명'].unique()


array(['자산[abstract]', '유동자산', '현금및현금성자산', ..., '매각예정처분집단의감소',
       '비현금항목및운전자본조정(주석32)', '단가차입금의감소'], dtype=object)

In [15]:
# 데이터 넣을 리스트 생성 : 20개 재무비율 생성을 위한 개별 항목 데이터 수집하기
data = []

# 재무비율계산에 필요한 항목명 가져오기
data_list = ["유동자산","유동부채","재고자산","자산총계","부채총계","자본총계","현금및현금성자산","당기순이익(손실)","매출총이익","매출액",
             "영업이익(손실)","영업이익","매출채권및기타유동채권","이자비용","영업활동현금흐름","영업활동으로인한현금흐름"]
# 데이터 추출
for i in data_list:
    item = df_fs[df_fs["항목명"] == i]
    data.append(item)

In [16]:
# 데이터 병합
df_fi = pd.concat(data).reset_index(drop=True)
df_fi   

,종목코드,회사명,시장구분,결산월,결산기준일,항목명,당기,전기
0,[060310],3S,코스닥시장상장법인,3,2015-03-31,유동자산,"14,907,177,686","16,826,799,889"
1,[095570],AJ네트웍스,유가증권시장상장법인,12,2015-12-31,유동자산,"91,870,888,881","37,214,915,600"
2,[068400],AJ렌터카,유가증권시장상장법인,12,2015-12-31,유동자산,"72,498,000,656","55,972,274,269"
3,[006840],AK홀딩스,유가증권시장상장법인,12,2015-12-31,유동자산,"3,956,856,041","2,795,801,603"
4,[054620],AP시스템,코스닥시장상장법인,12,2015-12-31,유동자산,"139,164,322,680","106,114,885,039"
...,...,...,...,...,...,...,...,...
230858,[298020],효성티앤씨,유가증권시장상장법인,12,2023-12-31,영업활동으로인한현금흐름,"272,973,565,817","242,349,505,650"
230859,[298000],효성화학,유가증권시장상장법인,12,2023-12-31,영업활동으로인한현금흐름,"125,492,496,520","-9,426,964,342"
230860,[005870],휴니드테크놀러지스,유가증권시장상장법인,12,2023-12-31,영업활동으로인한현금흐름,"17,511,904,228","48,128,563,271"
230861,[263920],휴엠앤씨,코스닥시장상장법인,12,2023-12-31,영업활동으로인한현금흐름,"6,141,778,559","-291,789,252"


In [17]:
# 데이터 통일을 위한 replace : 영업활동으로인한현금흐름"--> "영업활동현금흐름
df_fi["항목명"] = df_fi["항목명"].str.replace("영업활동으로인한현금흐름","영업활동현금흐름")
df_fi["항목명"] = df_fi["항목명"].str.replace("매출채권및기타유동채권","매출채권")
df_fi["항목명"] = df_fi["항목명"].str.replace("영업이익(손실)","영업이익")
df_fi["항목명"] = df_fi["항목명"].str.replace("당기순이익(손실)","순이익")
df_fi["항목명"] = df_fi["항목명"].str.replace("부채총계","총부채")
df_fi["항목명"] = df_fi["항목명"].str.replace("자산총계","총자산")
df_fi["항목명"] = df_fi["항목명"].str.replace("자본총계","자기자본")

In [18]:
# 재무지표 데이터 확인
df_fi["항목명"].unique()

array(['유동자산', '유동부채', '재고자산', '총자산', '총부채', '자기자본', '현금및현금성자산', '순이익',
       '매출총이익', '매출액', '영업이익', '매출채권', '이자비용', '영업활동현금흐름'], dtype=object)

In [19]:
# 데이터 저장
df_fi.to_csv("./1.2 재무데이터구분.csv",index=None)

### 2-2 재무비율 계산
- 각 기업 기업년도의 재무비율을 계산

In [20]:
# 결산년도별 정리: 재무비율 데이터프레임을 위한 각 년도별 고유한 값 확인
df_fi["결산년도"] = df_fi["결산기준일"].str[:4]
unique_values = df_fi[["종목코드", "회사명", "결산월", "결산년도"]].drop_duplicates()

# 고유한 값 출력
unique_values.reset_index(drop=True, inplace=True)
unique_values

,종목코드,회사명,결산월,결산년도
0,[060310],3S,3,2015
1,[095570],AJ네트웍스,12,2015
2,[068400],AJ렌터카,12,2015
3,[006840],AK홀딩스,12,2015
4,[054620],AP시스템,12,2015
...,...,...,...,...
19361,[064760],티씨케이,12,2023
19362,[033540],파라텍,12,2023
19363,[052600],한네트,12,2023
19364,[020000],한섬,12,2023


In [21]:
# 데이터 유형 변환: 당기와 전기 컬럼에서 숫자 데이터를 사용하기 위해 쉼표 제거 후 float형으로 변환
df_fi["당기"] = df_fi["당기"].str.replace(",", "").astype(float)
df_fi["전기"] = df_fi["전기"].str.replace(",", "").astype(float)

In [22]:
# 재무비율 계산 코드
def calculate_liquidity_ratio(df):
    
    liquidity_df_cy = df.pivot_table(index=["회사명", "결산년도"], columns="항목명", values="당기", aggfunc="first")
    liquidity_df_py = df.pivot_table(index=["회사명", "결산년도"], columns="항목명", values="전기", aggfunc="first")
    
    # NaN 값을 0으로 대체
    liquidity_df_cy = liquidity_df_cy.fillna(0)
    liquidity_df_py = liquidity_df_py.fillna(0)
    
    # 재무비율 계산 코드
    liquidity_df_cy["유동비율"] = (liquidity_df_cy["유동자산"]/  liquidity_df_cy["유동부채"]) * 100
    liquidity_df_cy["당좌비율"] = ((liquidity_df_cy["유동자산"] - liquidity_df_cy["재고자산"])/  liquidity_df_cy["유동부채"]) * 100
    liquidity_df_cy["부채비율"] = (liquidity_df_cy["총부채"]/  liquidity_df_cy["총자산"]) * 100
    liquidity_df_cy["자기자본비율"] = (liquidity_df_cy["자기자본"]/  liquidity_df_cy["총자산"]) * 100
    liquidity_df_cy["차입금의존도"] = (liquidity_df_cy["총부채"]/  liquidity_df_cy["자기자본"]) * 100
    liquidity_df_cy["현금비율"] = (liquidity_df_cy["현금및현금성자산"]/  liquidity_df_cy["유동부채"]) * 100
    liquidity_df_cy["순운전자본비율"] = ((liquidity_df_cy["유동자산"] - liquidity_df_cy["유동부채"])/  liquidity_df_cy["총자산"]) * 100
    
    liquidity_df_cy["자기자본이익률"] = (liquidity_df_cy["순이익"]/  (liquidity_df_cy["자기자본"]+liquidity_df_py["자기자본"])/2) * 100
    liquidity_df_cy["총자산이익률"] = (liquidity_df_cy["순이익"]/  (liquidity_df_cy["총자산"]+liquidity_df_py["총자산"])/2) * 100
    liquidity_df_cy["매출총이익률"] = (liquidity_df_cy["매출총이익"]/  liquidity_df_cy["매출액"]) * 100
    liquidity_df_cy["영업이익률"] = (liquidity_df_cy["영업이익"]/  liquidity_df_cy["매출액"]) * 100
    
    liquidity_df_cy["매출액성장률"] = (liquidity_df_cy["매출액"] - liquidity_df_py["매출액"])/  (liquidity_df_py["매출액"]) * 100
    liquidity_df_cy["자산성장률"] = (liquidity_df_cy["총자산"] - liquidity_df_py["총자산"])/  (liquidity_df_py["총자산"]) * 100
    liquidity_df_cy["자기자본성장률"] = (liquidity_df_cy["자기자본"] - liquidity_df_py["자기자본"])/  (liquidity_df_py["자기자본"]) * 100
    liquidity_df_cy["매출총이익성장률"] = (liquidity_df_cy["매출총이익"] - liquidity_df_py["매출총이익"])/  (liquidity_df_py["매출총이익"]) * 100
    
    liquidity_df_cy["총자산회전율"] = (liquidity_df_cy["매출액"]/  (liquidity_df_cy["총자산"]+liquidity_df_py["총자산"])/2) * 100
    liquidity_df_cy["매출채권회전율"] = (liquidity_df_cy["매출액"]/  (liquidity_df_cy["매출채권"]+liquidity_df_py["매출채권"])/2) * 100
    liquidity_df_cy["재고자산회전율"] = (liquidity_df_cy["매출액"]/  (liquidity_df_cy["재고자산"]+liquidity_df_py["재고자산"])/2) * 100
    liquidity_df_cy["순운전자본회전율"] = (liquidity_df_cy["매출액"]/  ((liquidity_df_cy["유동자산"] - liquidity_df_cy["유동부채"])+(liquidity_df_py["유동자산"] - liquidity_df_py["유동부채"])/2)) * 100 
    liquidity_df_cy["총부채상환능력비율"] = (liquidity_df_cy["영업활동현금흐름"]/  liquidity_df_cy["총부채"]) * 100
    
    # 결과 DataFrame을 원본 DataFrame에 병합
    liquidity_df_cy = liquidity_df_cy.loc[:,"유동비율":"총부채상환능력비율"].reset_index()
    
    # 소수점 두 번째 자리까지 반올림
    liquidity_df_cy = liquidity_df_cy.round(2)

    return liquidity_df_cy


unique_values = unique_values.merge(calculate_liquidity_ratio(df_fi), 
                                    on=["회사명", "결산년도"], 
                                    how="left")


In [23]:
# 재무비율 계산 데이터 확인
unique_values

,종목코드,회사명,결산월,결산년도,유동비율,당좌비율,부채비율,자기자본비율,차입금의존도,현금비율,...,영업이익률,매출액성장률,자산성장률,자기자본성장률,매출총이익성장률,총자산회전율,매출채권회전율,재고자산회전율,순운전자본회전율,총부채상환능력비율
0,[060310],3S,3,2015,69.46,61.73,43.30,56.70,76.36,32.40,...,-inf,NaN,-5.61,-11.74,-34.97,0.0,0.0,0.0,-0.0,1.02
1,[095570],AJ네트웍스,12,2015,41.21,37.25,57.20,0.00,inf,16.36,...,inf,NaN,26.45,NaN,NaN,0.0,NaN,0.0,-0.0,-10.57
2,[068400],AJ렌터카,12,2015,16.40,14.99,77.94,22.06,353.21,4.96,...,inf,NaN,15.49,8.55,NaN,0.0,NaN,0.0,-0.0,-6.11
3,[006840],AK홀딩스,12,2015,7.44,7.44,10.36,89.64,11.55,3.33,...,inf,NaN,6.40,3.80,NaN,0.0,NaN,NaN,-0.0,18.96
4,[054620],AP시스템,12,2015,104.10,76.63,59.10,40.90,144.51,3.59,...,inf,NaN,23.66,9.57,47.87,0.0,0.0,0.0,0.0,-20.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19361,[064760],티씨케이,12,2023,NaN,NaN,7.51,92.49,8.12,NaN,...,NaN,NaN,3.11,9.93,NaN,0.0,NaN,NaN,NaN,0.00
19362,[033540],파라텍,12,2023,NaN,NaN,47.59,52.41,90.82,inf,...,NaN,NaN,-8.06,-0.80,NaN,0.0,0.0,NaN,NaN,0.00
19363,[052600],한네트,12,2023,NaN,NaN,27.02,72.98,37.02,inf,...,NaN,NaN,4.08,2.31,NaN,0.0,NaN,NaN,NaN,0.00
19364,[020000],한섬,12,2023,NaN,NaN,18.77,81.23,23.10,inf,...,inf,NaN,2.49,4.46,-4.13,0.0,0.0,NaN,NaN,0.00


### 3. label 부여하기 

: 부실의 정의==> 상장폐지
- 부실(1)은 상장폐지 기준으로 사업보고서 제출이 최근연도에 없을 때 상장폐지라고 보고 label 부여
- 부실(상장폐지일) 직전년도 사업보고서 label을 1을 부여(단, 직전년도 사업보고서 없는경우 직직년도에 1 부여)
- 2023년 데이터는 삭제
- label 1이 부여된 회사 기업데이터는 1을 제외한 0데이터는 제거

In [24]:
# 회사별로 결산년도 리스트 생성 후 조건에 따라 label 부여
unique_values['label'] = 0 # 정상기업
# 각 회사명 최종년도 데이터를 1로 인코딩 
last_years = unique_values.groupby('회사명')['결산년도'].transform('max')
unique_values.loc[unique_values['결산년도'] == last_years, 'label'] = 1 # 부실기업

In [25]:
# 2023년도 데이터 제거 
unique_values = unique_values[unique_values["결산년도"]!="2023"].reset_index(drop=True)


In [26]:
# "label"이 1인 회사의 고유 회사명만 추출: 예시 2020년 부도기업의 경우 2019년데이터는 1이고, 이전 데이터는 삭제
defalut_companies = unique_values[unique_values["label"] == 1]["회사명"].unique()

# 원본 데이터프레임에서 부실기업 회사명만 필터링
filtered_values = unique_values[(unique_values["회사명"].isin(defalut_companies))]

# 부실기업의 부실 전년도 데이터 0으로 된 데이터 제거
drop_index = filtered_values[filtered_values['label'] == 0].index
unique_values.drop(drop_index,inplace=True)

# 데이터 인덱스 리셋
unique_values.reset_index(drop=True, inplace=True)

# 데이터 확인
unique_values

,종목코드,회사명,결산월,결산년도,유동비율,당좌비율,부채비율,자기자본비율,차입금의존도,현금비율,...,매출액성장률,자산성장률,자기자본성장률,매출총이익성장률,총자산회전율,매출채권회전율,재고자산회전율,순운전자본회전율,총부채상환능력비율,label
0,[060310],3S,3,2015,69.46,61.73,43.30,56.70,76.36,32.40,...,NaN,-5.61,-11.74,-34.97,0.00,0.0,0.00,-0.0,1.02,0
1,[095570],AJ네트웍스,12,2015,41.21,37.25,57.20,0.00,inf,16.36,...,NaN,26.45,NaN,NaN,0.00,NaN,0.00,-0.0,-10.57,0
2,[006840],AK홀딩스,12,2015,7.44,7.44,10.36,89.64,11.55,3.33,...,NaN,6.40,3.80,NaN,0.00,NaN,NaN,-0.0,18.96,0
3,[054620],AP시스템,12,2015,104.10,76.63,59.10,40.90,144.51,3.59,...,NaN,23.66,9.57,47.87,0.00,0.0,0.00,0.0,-20.90,0
4,[211270],AP위성통신,12,2015,809.29,745.58,10.76,89.24,12.05,424.75,...,5.49,15.53,30.17,19.01,23.21,inf,310.84,91.5,101.53,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14629,[002410],범양건영,12,2022,NaN,NaN,43.23,56.77,76.14,NaN,...,NaN,-8.76,-11.87,NaN,0.00,NaN,NaN,NaN,0.00,0
14630,[000650],천일고속,12,2022,NaN,NaN,51.47,48.53,106.06,NaN,...,NaN,-12.54,-25.92,NaN,0.00,NaN,NaN,NaN,0.00,0
14631,[064760],티씨케이,12,2022,NaN,NaN,13.25,86.75,15.27,NaN,...,NaN,23.31,22.79,NaN,0.00,NaN,NaN,NaN,0.00,0
14632,[null],한국코러스,12,2022,NaN,NaN,98.63,1.37,7183.41,NaN,...,NaN,-9.83,-94.71,NaN,0.00,NaN,NaN,NaN,0.00,0


In [27]:
# 데이터 갯수(행) 확인
unique_values["label"].value_counts()

label
0    13803
1      831
Name: count, dtype: int64

In [28]:
# 0과 1의 기업 수 확인
print(len(unique_values[unique_values["label"] == 0]["회사명"].unique()))
print(len(unique_values[unique_values["label"] == 1]["회사명"].unique()))

2265
830


In [29]:
# 1 기업(부실기업) : 831개와 830갸 차이
# 중앙오션이라는 기업이 2019년에 결산월이 6월에서 12월로 바뀌면서 2개의 데이터가 동시에 1로 부여된 상태에서
# 이 중앙오션 데이터의 경우 데이터 전처리 과정에서 결산월을 12월로 통일할 때 처리하여 1개 숫자 차이

In [30]:
# 데이터 저장
unique_values.to_csv("./1.3 데이터수집완료.csv")